## Limpieza tenis femenino

Librerias

In [92]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime

Funciones

In [93]:
def df_jugador(jugador, masc):

    filtro_jugador = (masc['jugadorA'] == jugador) | (masc['jugadorB'] == jugador)
    df_jugador = masc[filtro_jugador].copy()

    df_jugador_A = df_jugador[df_jugador['jugadorA'] == jugador].copy()
    df_jugador_B = df_jugador[df_jugador['jugadorB'] == jugador].copy()

    columnas_A = [col for col in df_jugador.columns if col.endswith('A')]
    columnas_B = [col for col in df_jugador.columns if col.endswith('B')]
    mapeo_columnas = {**{col_a: col_b for col_a, col_b in zip(columnas_A, columnas_B)},
                    **{col_b: col_a for col_a, col_b in zip(columnas_A, columnas_B)}}

    df_jugador_B_invertido = df_jugador_B.rename(columns=mapeo_columnas)

    df_final = pd.concat([df_jugador_A, df_jugador_B_invertido], ignore_index=True)

    if 'fecha' in df_final.columns:
        df_final = df_final.sort_values('fecha', ascending=False)

    return df_final

Lectura de datos

In [94]:
fem = pd.read_csv(".\\data\\data2\\wta_2025-06-15.csv")

print(len(fem))
fem.head(5)

12552


C:\Users\Equipo\AppData\Local\Temp\ipykernel_17092\3785118046.py:1: DtypeWarning: Columns (8,9,30,31,32,33,46,47,126,127) have mixed types. Specify dtype option on import or set low_memory=False.
  fem = pd.read_csv(".\\data\\data2\\wta_2025-06-15.csv")


,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,ano,acesA,acesB,...,porcentajeJuegosGanadosAlSaqueB,JuegosGanadosAlSaqueB,porcentajeJuegosGanadosAlRestoA,JuegosGanadosAlRestoA,porcentajeJuegosGanadosAlRestoB,JuegosGanadosAlRestoB,JuegosGanadosA,JuegosGanadosB,PuntosGanadosRedA,PuntosGanadosRedB
0,abu-dhabi,Bencic B.,Krueger A.,2,1,08.02. 14:10,no,2025,2,3,...,33%,(4/12),67%,(8/12),33%,(4/12),(16/24),(8/24),NaN,NaN
1,abu-dhabi,Rybakina E.,Bencic B.,1,2,07.02. 16:10,no,2025,6,2,...,79%,(11/14),21%,(3/14),29%,(4/14),(13/28),(15/28),NaN,NaN
2,abu-dhabi,Krueger A.,Noskova L.,2,0,07.02. 14:10,no,2025,6,4,...,73%,(8/11),27%,(3/11),18%,(2/11),(12/22),(10/22),NaN,NaN
3,abu-dhabi,Linette M.,Noskova L.,0,2,06.02. 17:20,no,2025,4,3,...,89%,(8/9),11%,(1/9),40%,(4/10),(7/19),(12/19),NaN,NaN
4,abu-dhabi,Rybakina E.,Jabeur O.,2,1,06.02. 14:50,no,2025,3,1,...,73%,(11/15),27%,(4/15),20%,(3/15),(16/30),(14/30),NaN,NaN


In [95]:
fem['acesA'].unique()

array(['2', '6', '4', '3', '1', '0', '7', '8', '5', nan, '12', '11', '9',
       '14', '01.06.2025 17:14', '10', '13', '17', '15', '16',
       '30.05.2025 19:12', '20', '22', '18', 1.0, 0.0, 12.0, 3.0, 2.0,
       4.0, 6.0, 5.0, 8.0, 10.0, 7.0, 9.0], dtype=object)

In [96]:
jugadores_A = fem['jugadorA'].unique().tolist()
jugadores_B = fem['jugadorB'].unique().tolist()
todos_jugadores = list(set(jugadores_A + jugadores_B))
masc = fem[~fem['setsA'].isin(todos_jugadores)]

fem = fem[fem["setsA"].isin(['0', '1', '2'])]
fem = fem[fem["setsB"].isin(['0', '1', '2'])]
fem = fem[~fem['acesA'].isin(['30.05.2025 19:12', '01.06.2025 17:14'])]
fem['setsA'] = fem['setsA'].astype(int)
fem['setsB'] = fem['setsB'].astype(int)

filtroGrandSlam = (fem['torneo'] == 'australian-open') | (fem['torneo'] == 'us-open') | (fem['torneo'] == 'wimbledon') | (fem['torneo'] == 'french-open')
filtro3sets = (fem['setsA'] == 3) | (fem['setsB'] == 3)
filtro2sets = (fem['setsA'] == 2) | (fem['setsB'] == 2)
filtroGrandSlamGanaA = fem['setsA'] == 3
filtroGrandSlamGanaB = fem['setsB'] == 3
filtroNoGrandSlamGanaA = fem['setsA'] == 2
filtroNoGrandSlamGanaB = fem['setsB'] == 2

# Condición 1: es Grand Slam y no se llegó a 3 sets (partido incompleto/retirado)
cond1 = filtroGrandSlam & (~filtro3sets)
fem.loc[cond1, 'retirado'] = 'si'

# Condición 2: no es Grand Slam y no se llegó a 2 sets (partido incompleto/retirado)
cond2 = (~filtroGrandSlam) & (~filtro2sets)
fem.loc[cond2, 'retirado'] = 'si'

#Creo la variable superficie
torneos_superficies = [
    ('acapulco', 'Hard'),('adelaide', 'Hard'),('adelaide-2', 'Hard'),('almaty', 'Clay'),('amersfoort', 'Clay'),('amsterdam', 'Clay'),('antalya', 'Grass'),('antwerp', 'Hard'),('antwerp-2', 'Hard'),('asian-games', 'Hard'),
    ('astana', 'Hard'),('athens', 'Clay'),('atlanta', 'Hard'),('atp-cup', 'Hard'),('auckland', 'Hard'),('australian-open', 'Hard'),('bangkok', 'Hard'),('banja-luka', 'Clay'),('barcelona', 'Clay'),('basel', 'Hard'),
    ('bastad', 'Clay'),('beijing', 'Hard'),('belgrade', 'Clay'),('belgrade-2', 'Clay'),('berlin', 'Grass'),('bermuda', 'Hard'),('birmingham', 'Grass'),('bogota', 'Clay'),('bologna', 'Clay'),('bolzano', 'Hard'),
    ('bordeaux', 'Clay'),('boston', 'Hard'),('brasilia', 'Clay'),('brighton', 'Hard'),('brisbane', 'Hard'),('bucharest', 'Clay'),('budapest', 'Clay'),('buenos-aires', 'Clay'),('buzios', 'Clay'),('cagliari', 'Clay'),
    ('casablanca', 'Clay'),('chengdu', 'Hard'),('chennai', 'Hard'),('chicago', 'Hard'),('cincinnati', 'Hard'),('cologne', 'Hard'),('cologne-2', 'Hard'),('copenhagen', 'Hard'),('cordoba', 'Clay'),('costa-do-sauipe', 'Clay'),
    ('dallas', 'Hard'),('delray-beach', 'Hard'),('doha', 'Hard'),('dubai', 'Hard'),('dusseldorf', 'Clay'),('eastbourne', 'Grass'),('essen', 'Hard'),('estoril', 'Clay'),('florence', 'Hard'),('french-open', 'Clay'),
]

superficie_dict = dict(torneos_superficies)
fem['superficie'] = fem['torneo'].map(superficie_dict)

fem.loc[fem['torneo'] == 'memphis', 'ano'] = 2017
fem.loc[fem['torneo'] == 'merano', 'ano'] = 1997
fem.loc[fem['torneo'] == 'metz', 'ano'] = 2024
fem[['dia', 'hora']] = fem['fecha'].str.split(' ', expand=True) #pongo la fecha como de normal (13/01/2025)
fem['dia'] = fem['dia'].str.replace('.', '/', regex=False)
fem['ano'] = fem['ano'].apply(lambda x: str(int(x)) if pd.notna(x) else x)
fem['dia'] = fem['dia'].astype(str).str.strip()
fem['ano'] = fem['ano'].astype(str).str.strip()
fem['fecha'] = fem['dia'] + fem['ano']
fem = fem.drop(columns = ['dia', 'cuotaA', 'cuotaB'])
fem['fecha'] = pd.to_datetime(fem['fecha'], format='%d/%m/%Y')

filtro = (fem['fecha'].dt.year == 2025) & (fem['fecha'].dt.month == 12)
fem.loc[filtro, 'fecha'] = fem.loc[filtro, 'fecha'] - pd.DateOffset(years=1)

fem['sets'] = fem['setsA'] + fem['setsB']

#Me quedo con las cuotas buenas
fem = fem.rename(columns={
    'cuotaA_2': 'cuotaA',
    'cuotaB_2': 'cuotaB'
})

#Le quito el símbolo del porcentaje a las columnas que lo tienen
cols_porcentaje = [col for col in fem.columns if 'porcentaje' in col.lower()]
for col in cols_porcentaje:
    fem[col] = fem[col].astype(str).str.replace('%', '', regex=False)
    fem[col] = pd.to_numeric(fem[col], errors='coerce')

#Le quito los paréntesis a las columnas que lo tienen
cols_con_par = [col for col in fem.columns if fem[col].astype(str).str.contains(r'\(\d+/\d+\)').any()]
for col in cols_con_par:
    fem[col] = fem[col].astype(str).str.replace('(', '', regex=False).str.replace(')', '', regex=False)
    
#Creo la columna ganador

fem['ganador'] = -1

ganadorA1 = filtroGrandSlam & filtroGrandSlamGanaA
fem.loc[ganadorA1, 'ganador'] = 0

ganadorB1 = filtroGrandSlam & filtroGrandSlamGanaB
fem.loc[ganadorB1, 'ganador'] = 1

ganadorA2 = (~filtroGrandSlam) & filtroNoGrandSlamGanaA
fem.loc[ganadorA2, 'ganador'] = 0

ganadorB2 = (~filtroGrandSlam) & filtroNoGrandSlamGanaB
fem.loc[ganadorB2, 'ganador'] = 1

fem = fem[fem['acesA'] != '31.05.2025 00:16']
fem = fem.sort_values('fecha', ascending=False).reset_index(drop=True)

fem.head()

,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,ano,acesA,acesB,...,porcentajeJuegosGanadosAlRestoB,JuegosGanadosAlRestoB,JuegosGanadosA,JuegosGanadosB,PuntosGanadosRedA,PuntosGanadosRedB,superficie,hora,sets,ganador
0,grado,Salkova D.,Palicova B.,0,2,2025-06-14,no,2025,1,1,...,56.0,5/9,6/18,12/18,nan,nan,NaN,14:05,2,1
1,valencia,Vandewinkel H.,Chirico L.,0,2,2025-06-14,no,2025,6.0,5.0,...,36.0,4/11,10/22,12/22,nan,nan,NaN,17:05,2,1
2,grado,Valentova T.,Marcinko P.,2,0,2025-06-14,no,2025,5,0,...,29.0,2/7,12/15,3/15,nan,nan,NaN,17:45,2,0
3,ilkley,Jovic I.,Golubic V.,2,0,2025-06-14,no,2025,0,1,...,0.0,0/8,12/15,3/15,nan,nan,NaN,12:20,2,0
4,ilkley,Marino R.,Naef C.,2,1,2025-06-14,no,2025,7,5,...,8.0,1/13,16/27,11/27,nan,nan,NaN,14:45,3,0


In [97]:
jugador = "Badosa P."

df = df_jugador(jugador, fem)

display(df)

,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,ano,acesA,acesB,...,porcentajeJuegosGanadosAlRestoB,JuegosGanadosAlRestoB,JuegosGanadosA,JuegosGanadosB,PuntosGanadosRedA,PuntosGanadosRedB,superficie,hora,sets,ganador
42,french-open,Badosa P.,Kasatkina D.,0,2,2025-05-31,si,2025,0,2,...,50.0,5/10,6/19,13/19,6/9,4/5,Clay,11:10,2,-1
43,french-open,Badosa P.,Ruse G.,2,1,2025-05-29,si,2025,5,8,...,33.0,5/15,15/29,14/29,4/7,13/17,Clay,16:20,3,-1
44,french-open,Badosa P.,Osaka N.,2,1,2025-05-26,si,2025,4,7,...,13.0,2/15,18/29,11/29,7/8,6/11,Clay,14:00,3,-1
0,strasbourg,Badosa P.,Samsonova L.,1,2,2025-05-22,no,2025,3,5,...,33.0,5/15,14/29,15/29,nan,nan,NaN,16:30,3,1
1,strasbourg,Badosa P.,Bouzkova M.,0,0,2025-05-20,si,2025,0,2,...,67.0,2/3,3/6,3/6,nan,nan,NaN,13:10,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,moscow-2,Badosa P.,Danilovic O.,2,1,2018-07-23,no,2018,NaN,NaN,...,NaN,nan,nan,nan,nan,nan,NaN,10:20,3,0
82,rio-de-janeiro,Badosa P.,Duque-Marino M.,1,2,2016-02-14,no,2016,NaN,NaN,...,NaN,nan,nan,nan,nan,nan,NaN,13:10,3,0
40,rio-de-janeiro,Badosa P.,Buzarnescu M.,2,1,2016-02-13,no,2016,NaN,NaN,...,NaN,nan,nan,nan,nan,nan,NaN,15:45,3,0
83,antwerp,Badosa P.,Volodko K.,0,2,2015-02-08,no,2015,NaN,NaN,...,NaN,nan,nan,nan,nan,nan,Hard,13:40,2,0


In [98]:
fem['acesA'] = pd.to_numeric(fem['acesA'], errors='coerce')
fem['acesA'].mean()

np.float64(2.386861313868613)